<h1>Sexism Dataset in catalan</h1>


In [ ]:
!pip install -U sklearn
!pip install -U nltk
!pip install stopwordsiso

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
from pathlib import Path
import nltk
import re
from tqdm import tqdm
from nltk.corpus import stopwords
import pandas as pd
import stopwordsiso as stopwords
import snowballstemmer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import metrics
from sklearn.model_selection import cross_val_score

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Processing the data

In [ ]:
# processing the data
def load_data(filepath):
  data = pd.read_csv(filepath, sep='\t')
  X, y = [], []
  X = list(data['text'])
  y1 = list(data['task1'])
  y2 = list(data['task2'])

  return X, y1, y2

In [ ]:
print('Load data...')
Xtrain_, y1train, y2train = load_data('train.tsv')
Xtest_, y1test, y2test = load_data('test.tsv')

print('preprocess...')
stemmer = snowballstemmer.stemmer('catalan')

stop_words = stopwords.stopwords('ca')

def preprocess(text):
    text = text.lower()
    text = re.sub('@\S+', '', text) # Remove usernames
    text = re.sub('http[s]?://\S+', '', text) # Remove urls
    text  = re.sub('[\d\W_]+', ' ', text) # Remove numbers, emojis, simbols, ...
    text = text.split()
    text = [x for x in text if x not in stop_words]  # Eliminate the stopwords
    text = stemmer.stemWords(text) # Apply stemming
    return ' '.join(text)

Xtrain = []
for text in Xtrain_:
    Xtrain.append(preprocess(text))

print('Text: ', Xtrain[0])
print('Label 1:', y1train[0], 'Label 2:', y2train[0])

Xtest = []
for text in Xtest_:
    Xtest.append(preprocess(text))

print('Text: ', Xtest[0])
print('Label 1:', y1test[0], 'Label 2:', y2test[0])

Load data...
preprocess...
Text:  el s autoanomen ant feminaz merd coment vil ciutad gran responsab tu sach much ghan bawr bawr hai bey
Label 1: sexist Label 2: ideological-inequality
Text:  represent l pennsilvan s horror politic inic mov johnson harr of par jesu deu pais principal crist res deu jesu escandol sarc coneg estrateg moosl jug cart vict demand genoll s inclin
Label 1: non-sexist Label 2: non-sexist


In [ ]:
# Listing all stopwords used in this application
print(stop_words)

{'com', 'durant', 'tinc', 'saber', 'que', 'tantes', 'quina', 'en', 'llavors', 'mon', 'ha', 'oh', 'fas', 'pocs', 'soc', 'tot', 'érem', 'poder', 'bastant', 'sabeu', 'molta', 'te', 'després', 'sou', 'teves', 'un', 'ma', 'malgrat', 'pas', 'cadascuns', 'nostres', 'contra', 'els', 'altre', 'me', 'mons', 'cada', 'algunes', 'inclòs', "t'han", 'tenir', 'a', 'una', 'molts', 'encara', 'son', 'teu', 'vostre', 'tons', 'uns', 'tots', 'conseguim', 'estic', 'baix', "d'unes", 'et', 'haver', 'mode', 'estaven', 'he', 'hi', 'només', 'i', 'puc', 'qui', 'es', 'meus', 'perquè', 'ne', 'des de', 'quelcom', 'quines', 'per', 'va', 'consegueixo', 'ans', 'poca', 'també', 'estàvem', 'tenim', 'des', 'heu', 'sense', 'això', 'potser', 'vosaltres', 'últim', 'ambdós', 'elles', 'la', 'seus', 'sóc', 'hem', 'poques', 'aquells', 'eres', 'saps', 'mentre', 'del', 'aleshores', 'les', "d'uns", 'fora', 'qual', 'saben', 'sobretot', "t'he", 'li', 'sap', 'sobre', 'el', 'vostra', 'has', 'podeu', 'mateixa', 'eh', 'seves', 'poc', 'ton

In [ ]:
# Showing the first 5 original and processed sentences
for i in range(5):
  print('Text original: ', Xtrain_[i])
  print('Text processed:', Xtrain[i])

Text original:  Ella s'autoanomena "anti-feminazi", què tal la merda amb el teu comentari vil sobre un ciutadà gran responsable tu sach much ghani bawri-bawri hai bey https://t.co/ZMxTDwsY5D
Text processed: el s autoanomen ant feminaz merd coment vil ciutad gran responsab tu sach much ghan bawr bawr hai bey
Text original:  Ara, tornem a aquestes dones, les valentes i les belles, @Clare_Crawley i @tayshia. Aquestes nenes dolents es mereixen tant de crèdit per com ha anat aquesta temporada. Com a dona, he après molt d'ells i em sento més empoderada per esperar més en futures relacions.
Text processed: ar torn don valent bel nen dol mer cred an tempor don apr d sent empoder esper futur relacion
Text original:  @CurvyBandida @Xalynne_B Vaja, la teva faldilla és molt curta. Quina és la seva longitud? 5 polzades o més?
Text processed: vaj fald curt long polz
Text original:  @AurelieGuiboud Increïble! Preciós! Però em vaig riure molt quan vaig llegir sobre tu a la deriva a la teva cadira de r

# Task 1

In [ ]:
# Adding a count vectorizer and feature selection to the following Pipeline

num_features=70
count_vectorizer = CountVectorizer()
feature_selector= SelectKBest(chi2, k=num_features)

text_clf = Pipeline([
    # add count vetorizer
    ('vec', count_vectorizer),
    # add feature selector
    ('fs', feature_selector),

    ('clf', KNeighborsClassifier()),
])
text_clf.fit(Xtrain, y1train)

Pipeline(steps=[('vec', CountVectorizer()),
                ('fs',
                 SelectKBest(k=70,
                             score_func=<function chi2 at 0x7ff3697a68c0>)),
                ('clf', KNeighborsClassifier())])

In [ ]:
predicted = text_clf.predict(Xtest)

In [ ]:
text_clf2 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', KNeighborsClassifier()),
])
text_clf2.fit(Xtrain, y1train)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', KNeighborsClassifier())])

In [ ]:
predicted2 = text_clf2.predict(Xtest)

In [ ]:
text_clf3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier()),
])
text_clf3.fit(Xtrain, y1train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', KNeighborsClassifier())])

In [ ]:
predicted3 = text_clf3.predict(Xtest)

# Evaluation

In [ ]:
# Evaluation
label_names = ['sexist', 'non-sexist']
print(metrics.classification_report(y1test, predicted,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y1test, predicted))

              precision    recall  f1-score   support

      sexist       0.60      0.84      0.70      2087
  non-sexist       0.77      0.48      0.59      2281

    accuracy                           0.66      4368
   macro avg       0.68      0.66      0.65      4368
weighted avg       0.69      0.66      0.64      4368

Confusion matrix
[[1759  328]
 [1178 1103]]


In [ ]:
label_names = ['sexist', 'non-sexist']
print(metrics.classification_report(y1test, predicted2,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y1test, predicted2))

              precision    recall  f1-score   support

      sexist       0.54      0.61      0.57      2087
  non-sexist       0.59      0.51      0.55      2281

    accuracy                           0.56      4368
   macro avg       0.56      0.56      0.56      4368
weighted avg       0.56      0.56      0.56      4368

Confusion matrix
[[1278  809]
 [1109 1172]]


In [ ]:
label_names = ['sexist', 'non-sexist']
print(metrics.classification_report(y1test, predicted3,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y1test, predicted3))

              precision    recall  f1-score   support

      sexist       0.59      0.36      0.44      2087
  non-sexist       0.57      0.78      0.66      2281

    accuracy                           0.58      4368
   macro avg       0.58      0.57      0.55      4368
weighted avg       0.58      0.58      0.55      4368

Confusion matrix
[[ 742 1345]
 [ 511 1770]]


In [ ]:
# Cross-validation
scores = cross_val_score(text_clf, Xtrain+Xtest, y1train+y1test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.66911059 0.62994432 0.52423864 0.64518192 0.63891688 0.61368767
 0.55052097 0.63893847 0.63946867 0.65215131]
Avg score:  0.6202159432498078
std 0.04394436516065604


In [ ]:
scores = cross_val_score(text_clf2, Xtrain+Xtest, y1train+y1test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.56033058 0.57053973 0.55066044 0.59908347 0.55927955 0.51058698
 0.54668645 0.5467022  0.56253648 0.54660706]
Avg score:  0.5553012954098756
std 0.02118304543561232


In [ ]:
scores = cross_val_score(text_clf3, Xtrain+Xtest, y1train+y1test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.41678599 0.47467622 0.47029137 0.49651244 0.47092943 0.39004528
 0.44078977 0.44468924 0.45324269 0.47095672]
Avg score:  0.45289191364865367
std 0.02966301057732842


# Task 2

In [ ]:
# Adding a count vectorizer and feature selection to the following Pipeline

num_features=50
count_vectorizer = CountVectorizer()
feature_selector= SelectKBest(chi2, k=num_features)

text_clf = Pipeline([
    # add count vetorizer
    ('vec', count_vectorizer),
    # add feature selector
    ('fs', feature_selector),

    ('clf', KNeighborsClassifier()),
])
text_clf.fit(Xtrain, y2train)

Pipeline(steps=[('vec', CountVectorizer()),
                ('fs',
                 SelectKBest(k=50,
                             score_func=<function chi2 at 0x7ff3697a68c0>)),
                ('clf', KNeighborsClassifier())])

In [ ]:
predicted = text_clf.predict(Xtest)

In [ ]:
text_clf2 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', KNeighborsClassifier()),
])
text_clf2.fit(Xtrain, y2train)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', KNeighborsClassifier())])

In [ ]:
predicted2 = text_clf2.predict(Xtest)

In [ ]:
text_clf3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', KNeighborsClassifier()),
])
text_clf3.fit(Xtrain, y2train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', KNeighborsClassifier())])

In [ ]:
predicted3 = text_clf3.predict(Xtest)

# Evaluation

In [ ]:
# Evaluation
label_names = ['ideological-inequality', 'stereotyping-dominance', 'objectification', 'sexual-violence', 'misogyny-non-sexual-violence', 'non-sexist']
print(metrics.classification_report(y2test, predicted,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y2test, predicted))

                              precision    recall  f1-score   support

      ideological-inequality       0.62      0.48      0.54       621
      stereotyping-dominance       0.33      0.25      0.29       472
             objectification       0.58      0.85      0.69      2087
             sexual-violence       0.41      0.22      0.29       324
misogyny-non-sexual-violence       0.65      0.20      0.30       400
                  non-sexist       0.51      0.16      0.25       464

                    accuracy                           0.56      4368
                   macro avg       0.52      0.36      0.39      4368
                weighted avg       0.54      0.56      0.51      4368

Confusion matrix
[[ 297   24  273    6    5   16]
 [  36  120  291    9   10    6]
 [  78  132 1784   48   19   26]
 [   8   44  181   72    2   17]
 [  27   24  237   26   79    7]
 [  35   21  312   13    7   76]]


In [ ]:
label_names = ['ideological-inequality', 'stereotyping-dominance', 'objectification', 'sexual-violence', 'misogyny-non-sexual-violence', 'non-sexist']
print(metrics.classification_report(y2test, predicted2,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y2test, predicted2))

                              precision    recall  f1-score   support

      ideological-inequality       0.58      0.13      0.22       621
      stereotyping-dominance       0.19      0.43      0.27       472
             objectification       0.52      0.68      0.59      2087
             sexual-violence       0.44      0.21      0.29       324
misogyny-non-sexual-violence       0.49      0.05      0.09       400
                  non-sexist       0.41      0.20      0.27       464

                    accuracy                           0.43      4368
                   macro avg       0.44      0.28      0.29      4368
                weighted avg       0.47      0.43      0.40      4368

Confusion matrix
[[  82   87  411    5    4   32]
 [   4  204  232   13    2   17]
 [  33  521 1422   40   12   59]
 [   8   71  158   69    4   14]
 [   5  102  253   12   21    7]
 [   9   76  271   17    0   91]]


In [ ]:
label_names = ['ideological-inequality', 'stereotyping-dominance', 'objectification', 'sexual-violence', 'misogyny-non-sexual-violence', 'non-sexist']
print(metrics.classification_report(y2test, predicted3,
     target_names=label_names))

print('Confusion matrix')
print(metrics.confusion_matrix(y2test, predicted3))

                              precision    recall  f1-score   support

      ideological-inequality       0.51      0.07      0.13       621
      stereotyping-dominance       0.12      0.78      0.21       472
             objectification       0.59      0.36      0.44      2087
             sexual-violence       0.55      0.06      0.10       324
misogyny-non-sexual-violence       0.71      0.01      0.02       400
                  non-sexist       0.87      0.04      0.08       464

                    accuracy                           0.27      4368
                   macro avg       0.56      0.22      0.17      4368
                weighted avg       0.57      0.27      0.27      4368

Confusion matrix
[[  46  406  169    0    0    0]
 [   2  366  104    0    0    0]
 [  30 1307  744    4    2    0]
 [   1  236   66   18    0    3]
 [   7  302   83    3    5    0]
 [   4  333   99    8    0   20]]


In [ ]:
# Cross-validation
scores = cross_val_score(text_clf, Xtrain+Xtest, y2train+y2test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.37010455 0.39631993 0.39116297 0.40478001 0.40811573 0.4079254
 0.35352852 0.33406791 0.39966279 0.42230475]
Avg score:  0.3887972562239684
std 0.026241454090889283


In [ ]:
scores = cross_val_score(text_clf2, Xtrain+Xtest, y2train+y2test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.31233989 0.32322869 0.33105269 0.34745065 0.32532304 0.32194508
 0.28478137 0.30837294 0.3187846  0.32582343]
Avg score:  0.3199102381922624
std 0.015452407751221506


In [ ]:
scores = cross_val_score(text_clf3, Xtrain+Xtest, y2train+y2test, cv=10, scoring='f1_macro', n_jobs=4)
print("scores: ", scores)
print("Avg score: ", scores.mean())
print("std", scores.std())

scores:  [0.14132435 0.23688923 0.21907343 0.26916023 0.21650763 0.19689404
 0.17577641 0.18398082 0.19469913 0.21272557]
Avg score:  0.2047030832959182
std 0.033233981312971755
